# Exercice 3

In [2]:
using JuMP, Gurobi, DataFrames, CSV, Random, LinearAlgebra, PyPlot, Dates, Statistics

# Question a

In [3]:
function RLR(y, X)
   
    n,m = size(X)
    model = Model(solver = GurobiSolver(TimeLimit=45))
    
    @variable(model, β[1:m])
    
    @objective(model, Min, sum((y - X*β).^2))
    
    sol = solve(model)
    
    return (getobjectivevalue(model), getvalue(β))
    
end

RLR (generic function with 1 method)

## Question b

In [4]:
sparseX2 = CSV.read("sparseX2.csv", header = false)
sparseY2 = CSV.read("sparseY2.csv", header = false)

,Column1
,Float64
1,2.34286
2,-3.04512
3,-5.90616
4,0.467956
5,-4.52402
6,1.77557
7,-1.1383
8,-3.26992
9,5.39506


Creation of the validation, training and testing sets.

In [5]:
function split_data(X, y, val, test)
    n = size(X, 1)
    index = shuffle([i for i in 1:n])
    
    size_validation = floor(Int,val*n)    
    size_test = floor(Int,(val+test)*n)
    
    ind_validation = index[1:size_validation]
    ind_test = index[size_validation+1:size_test]
    ind_train = index[size_test+1:n]
    
    X_validation = X[ind_validation, :]
    X_test = X[ind_test, :]
    X_train = X[ind_train, :]

    y_validation = y[ind_validation, :]
    y_test = y[ind_test, :]
    y_train = y[ind_train, :]
    
    return (
        convert(Matrix,X_validation), 
        convert(Matrix,X_test), 
        convert(Matrix,X_train), 
        convert(Matrix, y_validation), 
        convert(Matrix,y_test),
        convert(Matrix,y_train)
        )
    
end

split_data (generic function with 1 method)

In [6]:
(
    X_validation, 
    X_test, 
    X_train, 
    y_validation, 
    y_test,
    y_train
) = split_data(sparseX2, sparseY2, 0.25, 0.25)

([0.865359034508392 -1.3830239105437792 … -0.6321460869851878 0.2668197344511686; -0.5223367574215084 -1.0915556882219897 … 0.49415068848214566 -0.29532793316973055; … ; -0.0667679865065892 0.8739856083349014 … 2.007634258006536 1.6592842706869955; -0.6646462443910314 -1.513350928595663 … 2.164064096775709 -0.0924081803019496], [-1.9667458603144716 0.8667513992731141 … -1.7950553186304543 0.12001548936997525; -0.06337459242956062 -0.918403552104298 … -1.404343687682439 -1.2039277776012383; … ; -1.348533456491827 -1.0562153109753674 … 1.1881142311749096 -0.3399704288111639; 0.7973041601753904 -0.5232908450257276 … 0.12064798321850932 2.280177984189599], [-0.9064866353733989 -1.774970345560557 … 1.089429308879839 -0.4470390677875259; 1.2873446803519781 -1.2318226776506749 … 1.382633107897221 0.1596619727622882; … ; 1.2497099131508578 0.8644553636016384 … -2.850487575621966 -0.5971027475776414; -0.43870997575879667 0.44237536440754566 … 0.2869531210793497 -0.3959998081620423], [-0.0676436

We choose ρ as the value that gives the best mean squared prediction error on the validation set.

In [7]:
function mean_square_error(y_actual, y_predicted)
    return sum((y_actual - y_predicted).^2)
end

mean_square_error (generic function with 1 method)

In [8]:
function compute_error(X, y)
    objective, β = RLR(y_train, X_train)
    error = mean_square_error(y, X * β)
    return error
end

compute_error (generic function with 1 method)

In [9]:
SLR_error = compute_error(X_test, y_test)

Academic license - for non-commercial use only
Optimize a model with 0 rows, 100 columns and 0 nonzeros
Model has 5050 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 3e+02]
  QObjective range [9e-03, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 100 columns, 0 nonzeros
Presolved model has 5050 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 199
 AA' NZ     : 4.851e+03
 Factor NZ  : 4.950e+03
 Factor Ops : 3.284e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   0.00000000e+00  0.00000000e+00  0.00e+00 3.33e+02  0.00e+00     0s
   1  -2.59846287e+02 -1.57837289e+01  3.77e-08 2.95e+02  0.00e+00     0s
   2  -4.95692372e+02 -6.54093558e+01  3.24e-08 2.55e+02  0.00e+00     0s
   3  -7.44552

6391.2302259375865

In [13]:
function baseline_error(y, data)
    avg = mean(data)
    error = sum((y .- avg).^2)
    return(error)
end

baseline_error (generic function with 1 method)

In [14]:
baseline_e = baseline_error(y_test, convert(Matrix,sparseY2))

611.0760445152437

In [15]:
println(" SLR error: ", SLR_error)
println(" baseline error: ", baseline_e)

 SLR error: 6391.2302259375865
 baseline error: 611.0760445152437
